### Import libraries

In [1]:
import pandas as pd
import os
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error

### Set the data repository as the working directory 

In [2]:
print(os.path.abspath(os.curdir))

C:\Users\Asus\Documents\College\UP - MS EE\SY 2022-2023 Second Semester (temporary, final files in new laptop)\EE 290\Files (NEW May 31) TG 1 2 3\codes


In [3]:
os.chdir("../data")
print(os.path.abspath(os.curdir))

C:\Users\Asus\Documents\College\UP - MS EE\SY 2022-2023 Second Semester (temporary, final files in new laptop)\EE 290\Files (NEW May 31) TG 1 2 3\data


### Import the test set (increase in load)

##### Test set (increase in load)

In [4]:
pd_qd_test_load_increase = pd.read_csv('pd_qd_test_load_increase.csv')
pd_qd_test_load_increase.head()

,scenario,pd_2,qd_2,pd_3,qd_3,pd_4,qd_4,pd_5,qd_5,pd_6,...,pd_29,qd_29,pd_30,qd_30,pd_31,qd_31,pd_32,qd_32,pd_33,qd_33
0,1,0.102095,0.021776,0.089378,0.004000,0.117751,0.058026,0.058975,0.032789,0.058938,...,0.118753,0.087211,0.197275,0.112329,0.147804,0.044803,0.213795,0.089132,0.061060,0.030773
1,2,0.098302,0.017799,0.089865,0.065295,0.117826,0.067090,0.060564,0.021982,0.060701,...,0.120286,0.071373,0.200025,0.138059,0.149505,0.092262,0.209256,0.040249,0.060760,0.037496
2,3,0.102045,0.053405,0.091226,0.017547,0.118231,0.067638,0.059846,0.012462,0.061008,...,0.119919,0.082146,0.203486,0.090325,0.147832,0.070759,0.211334,0.054787,0.059075,0.018565
3,4,0.098905,0.048181,0.089883,0.059231,0.122257,0.086289,0.059353,0.032839,0.058532,...,0.122064,0.085835,0.200333,0.075987,0.148417,0.088458,0.210222,0.128625,0.059598,0.037873
4,5,0.099208,0.056224,0.089409,0.061711,0.120511,0.066026,0.059441,0.032405,0.059714,...,0.119327,0.087632,0.198470,0.106042,0.147544,0.100686,0.208752,0.088297,0.061241,0.005494


In [5]:
pg_v_test_load_increase = pd.read_csv('pg_v_test_load_increase.csv')
pg_v_test_load_increase.head()

,scenario,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1,1.763048,0.499980,0.995820,0.499975,1.003092,0.499979,0.988517,0.499981,0.986707
1,2,1.737040,0.499980,0.995817,0.499975,1.002406,0.499979,0.987031,0.499982,0.986740
2,3,1.764525,0.499980,0.993074,0.499976,1.001879,0.499979,0.986218,0.499982,0.986738
3,4,1.757166,0.499975,0.992044,0.499971,1.002813,0.499974,0.984583,0.499978,0.983328
4,5,1.749419,0.499975,0.994683,0.499971,1.001771,0.499974,0.985633,0.499978,0.986264


### Separate the dependent variables (optimal generator setpoints) and the independent variables (non-dispatchable loads)
Non-dispatchable loads: $x = (P_{D_{i}},Q_{D_{i}})\forall i\in \Omega_{D}$
<br>
Optimal generator setpoints: $y = (P_{G_{j}},V_{j})\forall j\in \Omega_{G}$

##### Test set (increase in load)

In [6]:
x_test_load_increase = pd_qd_test_load_increase.loc[:, pd_qd_test_load_increase.columns != 'scenario']
x_test_load_increase.head()

,pd_2,qd_2,pd_3,qd_3,pd_4,qd_4,pd_5,qd_5,pd_6,qd_6,...,pd_29,qd_29,pd_30,qd_30,pd_31,qd_31,pd_32,qd_32,pd_33,qd_33
0,0.102095,0.021776,0.089378,0.004000,0.117751,0.058026,0.058975,0.032789,0.058938,0.028712,...,0.118753,0.087211,0.197275,0.112329,0.147804,0.044803,0.213795,0.089132,0.061060,0.030773
1,0.098302,0.017799,0.089865,0.065295,0.117826,0.067090,0.060564,0.021982,0.060701,0.027302,...,0.120286,0.071373,0.200025,0.138059,0.149505,0.092262,0.209256,0.040249,0.060760,0.037496
2,0.102045,0.053405,0.091226,0.017547,0.118231,0.067638,0.059846,0.012462,0.061008,0.017794,...,0.119919,0.082146,0.203486,0.090325,0.147832,0.070759,0.211334,0.054787,0.059075,0.018565
3,0.098905,0.048181,0.089883,0.059231,0.122257,0.086289,0.059353,0.032839,0.058532,0.028844,...,0.122064,0.085835,0.200333,0.075987,0.148417,0.088458,0.210222,0.128625,0.059598,0.037873
4,0.099208,0.056224,0.089409,0.061711,0.120511,0.066026,0.059441,0.032405,0.059714,0.012434,...,0.119327,0.087632,0.198470,0.106042,0.147544,0.100686,0.208752,0.088297,0.061241,0.005494


In [7]:
y_test_load_increase = pg_v_test_load_increase.loc[:, pg_v_test_load_increase.columns != 'scenario']
y_test_load_increase.head()

,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1.763048,0.499980,0.995820,0.499975,1.003092,0.499979,0.988517,0.499981,0.986707
1,1.737040,0.499980,0.995817,0.499975,1.002406,0.499979,0.987031,0.499982,0.986740
2,1.764525,0.499980,0.993074,0.499976,1.001879,0.499979,0.986218,0.499982,0.986738
3,1.757166,0.499975,0.992044,0.499971,1.002813,0.499974,0.984583,0.499978,0.983328
4,1.749419,0.499975,0.994683,0.499971,1.001771,0.499974,0.985633,0.499978,0.986264


### Test the trained models on the test set (increase in load)

In [8]:
model_1 = load_model('model_1.h5')
model_2 = load_model('model_2.h5')
model_3 = load_model('model_3.h5')

##### Using model 1 that was trained using training group 1

In [9]:
y_test_load_increase_pred_1 = model_1.predict(x_test_load_increase)
y_test_load_increase_pred_1 = pd.DataFrame(data = y_test_load_increase_pred_1,
                                 columns = ['pg_1', 'pg_18', 'v_18', 'pg_22', 'v_22', 'pg_25', 'v_25', 'pg_33', 'v_33'])
y_test_load_increase_pred_1.head()

,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1.758103,0.503435,0.994450,0.498228,1.001049,0.498079,0.989166,0.500742,0.983483
1,1.730381,0.502246,0.995173,0.499606,1.001341,0.499537,0.988894,0.500680,0.985199
2,1.756925,0.502944,0.992318,0.498320,1.000921,0.496838,0.986820,0.500379,0.982871
3,1.749948,0.500576,0.992066,0.497719,1.002545,0.495299,0.981771,0.499674,0.980809
4,1.744508,0.501333,0.993760,0.498580,1.001633,0.497185,0.985005,0.499587,0.983420


In [10]:
pg_v_test_load_increase_pred_1 = pd.concat([pg_v_test_load_increase.scenario, y_test_load_increase_pred_1], axis = 1)
pg_v_test_load_increase_pred_1.head()

,scenario,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1,1.758103,0.503435,0.994450,0.498228,1.001049,0.498079,0.989166,0.500742,0.983483
1,2,1.730381,0.502246,0.995173,0.499606,1.001341,0.499537,0.988894,0.500680,0.985199
2,3,1.756925,0.502944,0.992318,0.498320,1.000921,0.496838,0.986820,0.500379,0.982871
3,4,1.749948,0.500576,0.992066,0.497719,1.002545,0.495299,0.981771,0.499674,0.980809
4,5,1.744508,0.501333,0.993760,0.498580,1.001633,0.497185,0.985005,0.499587,0.983420


In [11]:
pg_v_test_load_increase_pred_1.to_csv('pg_v_test_load_increase_pred_1.csv', index = False, float_format = '%0.20f')

##### Using model 2 that was trained using training group 2

In [12]:
y_test_load_increase_pred_2 = model_2.predict(x_test_load_increase)
y_test_load_increase_pred_2 = pd.DataFrame(data = y_test_load_increase_pred_2,
                             columns = ['pg_1', 'pg_18', 'v_18', 'pg_22', 'v_22', 'pg_25', 'v_25', 'pg_33', 'v_33'])
y_test_load_increase_pred_2.head()

,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1.748833,0.459137,0.956699,0.549907,1.006145,0.477175,1.007524,0.465455,0.962834
1,1.715277,0.470319,0.954082,0.550982,1.003842,0.472772,1.012039,0.464849,0.963172
2,1.733594,0.468135,0.953403,0.548101,0.999925,0.468751,1.016086,0.467243,0.965595
3,1.736339,0.462264,0.955321,0.543926,1.000333,0.471739,1.014394,0.470404,0.965489
4,1.734857,0.456640,0.959846,0.542011,1.000623,0.473534,1.010645,0.472095,0.969233


In [13]:
pg_v_test_load_increase_pred_2 = pd.concat([pg_v_test_load_increase.scenario, y_test_load_increase_pred_2], axis = 1)
pg_v_test_load_increase_pred_2.head()

,scenario,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1,1.748833,0.459137,0.956699,0.549907,1.006145,0.477175,1.007524,0.465455,0.962834
1,2,1.715277,0.470319,0.954082,0.550982,1.003842,0.472772,1.012039,0.464849,0.963172
2,3,1.733594,0.468135,0.953403,0.548101,0.999925,0.468751,1.016086,0.467243,0.965595
3,4,1.736339,0.462264,0.955321,0.543926,1.000333,0.471739,1.014394,0.470404,0.965489
4,5,1.734857,0.456640,0.959846,0.542011,1.000623,0.473534,1.010645,0.472095,0.969233


In [14]:
pg_v_test_load_increase_pred_2.to_csv('pg_v_test_load_increase_pred_2.csv', index = False, float_format = '%0.20f')

##### Using model 3 that was trained using training group 3

In [15]:
y_test_load_increase_pred_3 = model_3.predict(x_test_load_increase)
y_test_load_increase_pred_3 = pd.DataFrame(data = y_test_load_increase_pred_3,
                             columns = ['pg_1', 'pg_18', 'v_18', 'pg_22', 'v_22', 'pg_25', 'v_25', 'pg_33', 'v_33'])
y_test_load_increase_pred_3.head()

,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1.478901,0.500282,1.106831,0.458472,0.969530,0.569854,0.910408,0.483546,0.956185
1,1.475747,0.496499,1.122425,0.459337,0.964005,0.575832,0.906824,0.484286,0.948981
2,1.507088,0.500255,1.120456,0.460823,0.962268,0.576094,0.904024,0.483789,0.956595
3,1.502123,0.501022,1.119619,0.459621,0.962419,0.576697,0.906796,0.476924,0.944342
4,1.483196,0.500567,1.123713,0.456131,0.960796,0.572214,0.905903,0.480459,0.949211


In [16]:
pg_v_test_load_increase_pred_3 = pd.concat([pg_v_test_load_increase.scenario, y_test_load_increase_pred_3], axis = 1)
pg_v_test_load_increase_pred_3.head()

,scenario,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1,1.478901,0.500282,1.106831,0.458472,0.969530,0.569854,0.910408,0.483546,0.956185
1,2,1.475747,0.496499,1.122425,0.459337,0.964005,0.575832,0.906824,0.484286,0.948981
2,3,1.507088,0.500255,1.120456,0.460823,0.962268,0.576094,0.904024,0.483789,0.956595
3,4,1.502123,0.501022,1.119619,0.459621,0.962419,0.576697,0.906796,0.476924,0.944342
4,5,1.483196,0.500567,1.123713,0.456131,0.960796,0.572214,0.905903,0.480459,0.949211


In [17]:
pg_v_test_load_increase_pred_3.to_csv('pg_v_test_load_increase_pred_3.csv', index = False, float_format = '%0.20f')

##### RMSE of $P_{G_{1}}$ estimates

In [18]:
mean_squared_error(pg_v_test_load_increase.pg_1, pg_v_test_load_increase_pred_1.pg_1, squared = False)

0.006560151143506911

In [19]:
mean_squared_error(pg_v_test_load_increase.pg_1, pg_v_test_load_increase_pred_2.pg_1, squared = False)

0.020355202883741826

In [20]:
mean_squared_error(pg_v_test_load_increase.pg_1, pg_v_test_load_increase_pred_3.pg_1, squared = False)

0.2677271599693111

##### RMSE of $P_{G_{18}}$ estimates

In [21]:
mean_squared_error(pg_v_test_load_increase.pg_18, pg_v_test_load_increase_pred_1.pg_18, squared = False)

0.002389220468878652

In [22]:
mean_squared_error(pg_v_test_load_increase.pg_18, pg_v_test_load_increase_pred_2.pg_18, squared = False)

0.04131787219800429

In [23]:
mean_squared_error(pg_v_test_load_increase.pg_18, pg_v_test_load_increase_pred_3.pg_18, squared = False)

0.003716763705923139

##### RMSE of $P_{G_{22}}$ estimates

In [24]:
mean_squared_error(pg_v_test_load_increase.pg_22, pg_v_test_load_increase_pred_1.pg_22, squared = False)

0.001712785449170211

In [25]:
mean_squared_error(pg_v_test_load_increase.pg_22, pg_v_test_load_increase_pred_2.pg_22, squared = False)

0.04638994385573285

In [26]:
mean_squared_error(pg_v_test_load_increase.pg_22, pg_v_test_load_increase_pred_3.pg_22, squared = False)

0.04316938592455001

##### RMSE of $P_{G_{25}}$ estimates

In [27]:
mean_squared_error(pg_v_test_load_increase.pg_25, pg_v_test_load_increase_pred_1.pg_25, squared = False)

0.0029019283005457756

In [28]:
mean_squared_error(pg_v_test_load_increase.pg_25, pg_v_test_load_increase_pred_2.pg_25, squared = False)

0.026560823532253724

In [29]:
mean_squared_error(pg_v_test_load_increase.pg_25, pg_v_test_load_increase_pred_3.pg_25, squared = False)

0.07318991769017699

##### RMSE of $P_{G_{33}}$ estimates

In [30]:
mean_squared_error(pg_v_test_load_increase.pg_33, pg_v_test_load_increase_pred_1.pg_33, squared = False)

0.0006493996014509167

In [31]:
mean_squared_error(pg_v_test_load_increase.pg_33, pg_v_test_load_increase_pred_2.pg_33, squared = False)

0.03095289313791558

In [32]:
mean_squared_error(pg_v_test_load_increase.pg_33, pg_v_test_load_increase_pred_3.pg_33, squared = False)

0.01831094261554384

##### RMSE of $V_{18}$ estimates

In [33]:
mean_squared_error(pg_v_test_load_increase.v_18, pg_v_test_load_increase_pred_1.v_18, squared = False)

0.0009225563754412099

In [34]:
mean_squared_error(pg_v_test_load_increase.v_18, pg_v_test_load_increase_pred_2.v_18, squared = False)

0.03731348756478039

In [35]:
mean_squared_error(pg_v_test_load_increase.v_18, pg_v_test_load_increase_pred_3.v_18, squared = False)

0.12519826997871764

##### RMSE of $V_{22}$ estimates

In [36]:
mean_squared_error(pg_v_test_load_increase.v_22, pg_v_test_load_increase_pred_1.v_22, squared = False)

0.0008104027477252237

In [37]:
mean_squared_error(pg_v_test_load_increase.v_22, pg_v_test_load_increase_pred_2.v_22, squared = False)

0.002388218899661204

In [38]:
mean_squared_error(pg_v_test_load_increase.v_22, pg_v_test_load_increase_pred_3.v_22, squared = False)

0.038624342621339784

##### RMSE of $V_{25}$ estimates

In [39]:
mean_squared_error(pg_v_test_load_increase.v_25, pg_v_test_load_increase_pred_1.v_25, squared = False)

0.0015860674262320443

In [40]:
mean_squared_error(pg_v_test_load_increase.v_25, pg_v_test_load_increase_pred_2.v_25, squared = False)

0.024934435465051703

In [41]:
mean_squared_error(pg_v_test_load_increase.v_25, pg_v_test_load_increase_pred_3.v_25, squared = False)

0.08019297171527454

##### RMSE of $V_{33}$ estimates

In [42]:
mean_squared_error(pg_v_test_load_increase.v_33, pg_v_test_load_increase_pred_1.v_33, squared = False)

0.0032416480568183905

In [43]:
mean_squared_error(pg_v_test_load_increase.v_33, pg_v_test_load_increase_pred_2.v_33, squared = False)

0.02004224383179839

In [44]:
mean_squared_error(pg_v_test_load_increase.v_33, pg_v_test_load_increase_pred_3.v_33, squared = False)

0.03188647362540422